# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data_mod6/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,291.40,96,76,1.21,ZA,1706672400
1,1,adamstown,-25.0660,-130.1015,298.77,76,100,11.95,PN,1706672400
2,2,puerto natales,-51.7236,-72.4875,281.48,86,55,1.70,CL,1706672400
3,3,polyarnyy,69.1989,33.4478,273.99,73,100,14.77,RU,1706672400
4,4,bucerias,20.7500,-105.3333,297.14,83,40,0.00,MX,1706672270
...,...,...,...,...,...,...,...,...,...,...
573,573,at taj,24.2049,23.2857,280.38,56,2,3.16,LY,1706672506
574,574,dilling,12.0500,29.6500,288.64,22,52,6.66,SD,1706672506
575,575,fale old settlement,-9.3852,-171.2468,302.89,68,26,1.34,TK,1706672506
576,576,kilindoni,-7.9139,39.6668,300.63,81,84,5.44,TZ,1706672506


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
humidity_plot_map = city_data_df.hvplot.points(x='Lng', y='Lat', size='Humidity', hover_cols=['City'], geo=True, colorbar=True, alpha=0.5, cmap='yellow', width=800, height=500,tiles='OSM')

# Display the map
humidity_plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[city_data_df["Max Temp"]>100][city_data_df["Max Temp"]<300][city_data_df["Wind Speed"] < 15][city_data_df["Country"] =="NZ"]
# Drop any rows with null values
city_data_df.dropna()
# Display sample data
ideal_city_df

C:\Users\KatreeceHattaway\AppData\Local\Temp\ipykernel_18072\1347516365.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_city_df = city_data_df[city_data_df["Max Temp"]>100][city_data_df["Max Temp"]<300][city_data_df["Wind Speed"] < 15][city_data_df["Country"] =="NZ"]
C:\Users\KatreeceHattaway\AppData\Local\Temp\ipykernel_18072\1347516365.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_city_df = city_data_df[city_data_df["Max Temp"]>100][city_data_df["Max Temp"]<300][city_data_df["Wind Speed"] < 15][city_data_df["Country"] =="NZ"]


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,waitangi,-43.9535,-176.5597,290.83,98,100,1.79,NZ,1706672318
51,51,invercargill,-46.4000,168.3500,292.60,49,98,4.02,NZ,1706672315
54,54,papatowai,-46.5619,169.4708,291.52,48,76,3.57,NZ,1706672411
85,85,christchurch,-43.5333,172.6333,294.79,61,3,10.80,NZ,1706672327
140,140,kerikeri,-35.2268,173.9474,295.94,69,65,2.15,NZ,1706672430
290,290,masterton,-40.9597,175.6575,296.42,54,82,1.65,NZ,1706672457
355,355,greymouth,-42.4667,171.2000,291.96,71,23,3.32,NZ,1706672467
502,502,gisborne,-38.6533,178.0042,293.31,81,67,5.54,NZ,1706672495


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,waitangi,NZ,-43.9535,-176.5597,98,
51,invercargill,NZ,-46.4000,168.3500,49,
54,papatowai,NZ,-46.5619,169.4708,48,
85,christchurch,NZ,-43.5333,172.6333,61,
140,kerikeri,NZ,-35.2268,173.9474,69,
290,masterton,NZ,-40.9597,175.6575,54,
355,greymouth,NZ,-42.4667,171.2000,71,
502,gisborne,NZ,-38.6533,178.0042,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame for the current row
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Define filters and bias based on the current hotel's location
    radius = 10000  # Adjust the radius as needed
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    # Set parameters for the API request
    params = {
        "categories": "accommodation.hotel",
        "apiKey": geoapify_key,
        "filter": filters,
        "bias": bias
    }

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)


Starting hotel search
waitangi - nearest hotel: Hotel Chathams
invercargill - nearest hotel: The Grand
papatowai - nearest hotel: No hotel found
christchurch - nearest hotel: Ibis Hotel Christchurch
kerikeri - nearest hotel: Avalon Resort
masterton - nearest hotel: Copthorne Hotel Solway Park
greymouth - nearest hotel: Recreation Hotel
gisborne - nearest hotel: Portside Hotel
             City Country      Lat       Lng  Humidity  \
27       waitangi      NZ -43.9535 -176.5597        98   
51   invercargill      NZ -46.4000  168.3500        49   
54      papatowai      NZ -46.5619  169.4708        48   
85   christchurch      NZ -43.5333  172.6333        61   
140      kerikeri      NZ -35.2268  173.9474        69   
290     masterton      NZ -40.9597  175.6575        54   
355     greymouth      NZ -42.4667  171.2000        71   
502      gisborne      NZ -38.6533  178.0042        81   

                      Hotel Name  
27                Hotel Chathams  
51                     The G

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
hotel_plot_map = hotel_df.hvplot.points(x='Lng', y='Lat', size='Humidity', hover_cols=['City', "Hotel Name"],
                                                geo=True, colorbar=True, alpha=0.5, cmap='Blues', width=800, height=500,tiles='OSM')
# Display the map
hotel_plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name)